# Step 2: Adding Tools to Your Agent

## The Problem

In Step 1, we saw that LLMs can only use their training knowledge. They can't:
- Get today's weather
- Check live stock prices
- Query a database
- Make API calls

## The Solution: Tools

**Tools** are functions that the LLM can call to take actions interact with the world.

**Reference:** [Pydantic AI Agents](https://ai.pydantic.dev/agents/)

## Setup

Let's import what we need. Notice we're now using `Agent` instead of the direct API.

In [1]:
# Note: This (nest_asyncio) is only required in Jupyter notebooks (https://ai.pydantic.dev/troubleshooting/)
import nest_asyncio

nest_asyncio.apply()

In [2]:
from pydantic_ai import Agent, RunContext
from pydantic import BaseModel
from tools import get_weather_for_city
import dotenv

dotenv.load_dotenv(override=True)

True

## Define User Context

Agents can be personalized. We'll create a `UserInfo` model to track:
- The user's name
- The user's city

This lets us build context-aware agents that remember who they're talking to.

In [3]:
class UserInfo(BaseModel):
    name: str
    city: str

## Create the Agent

Now we create an agent instead of making direct API calls.

**Key parameters:**
- `model`: Which LLM to use
- `deps_type`: What type of context/dependencies the agent needs (our `UserInfo`)

The agent will have access to user information whenever it runs.

In [4]:
basic_agent = Agent(
    model="anthropic:claude-3-5-haiku-latest",
    deps_type=UserInfo,
)

## Set the System Prompt

Here, we can use system prompt to tell the agent about the user through the dependency object `ctx.deps`.

In [5]:
@basic_agent.system_prompt
def set_system_prompt(ctx: RunContext[UserInfo]) -> str:
    return f"""
    You are a helpful assistant.
    Answer questions as best you can, using any tools as needed.
    The user's name is {ctx.deps.name} and they are in {ctx.deps.city}.
    """

## Register a Tool

This is where the magic happens. The `@basic_agent.tool` decorator registers a function as a tool the agent can call.

**Key parts of a tool:**
1. **Definition**
    - Pydantic AI will automatically generate a docstring from the function's signature and docstring
2. **Parameters**
3. **Return value**

The agent can **choose** whether to call this tool.

> Note: We've deliberately hidden the tool implementation in this example. It doesn't matter for the agent's decision-making. Take a look at the tool implementation in the `tools.py` file.

In [6]:
@basic_agent.tool
def get_weather(ctx: RunContext[UserInfo]) -> str:
    """Check the weather in a given location"""
    print(">> TOOL USED: Getting weather for user: ", ctx.deps)
    return get_weather_for_city(ctx.deps.city)

## Run the Agent

Let's test our agent.

**Watch for the "TOOL USED" message** - this tells us when the agent decided to call the weather tool!

In [7]:
model_response = basic_agent.run_sync(
    user_prompt="What's the weather like today where I am?",
    deps=UserInfo(name="JP", city="Edinburgh"),
)
print(model_response.output)

>> TOOL USED: Getting weather for user:  name='JP' city='Edinburgh'
It looks like it's a bit chilly and wet in Edinburgh today. The temperature is around 5°C (41°F), and it's raining. I recommend dressing warmly and bringing a waterproof jacket or umbrella if you're planning to go out. Stay dry, JP!


Try it again, with:
- Your own name and city
- A different prompt, not asking about the weather

In [8]:
# BEGIN_SOLUTION
model_response = basic_agent.run_sync(
    user_prompt="Tell me a punny joke about a chicken",
    deps=UserInfo(name="Daniel", city="Paris"),
)
print(model_response.output)
# END_SOLUTION

Here's a classic chicken joke for you:

Why did the chicken cross the playground? 

To get to the other slide! 

*Ba dum tss* 😄 I hope that brought a little chuckle to your day, Daniel! Sometimes a corny joke is just what we need to lighten the mood. Would you like to hear another punny chicken joke?


## What Just Happened?

Let's break down the agent's decision-making:

1. **User asks:** "What's the weather like today where I am?"
2. **Agent thinks:** "I need current weather data. I can't answer this from my training data alone."
3. **Agent decides:** "I should use the `get_weather` tool!"
4. **Tool executes:** Gets weather for the user's city
5. **Agent responds:** Uses the tool's result to give a helpful answer

## Key Takeaways

### What We Learned

✅ **Agents extend LLMs with tools** - Tools let LLMs access real-time data and take actions  
✅ **Agents make decisions** - The LLM decides *when* to use tools based on the question  
✅ **Context matters** - We can pass user information to personalize agent behavior  
✅ **Docstrings are critical** - The agent reads tool docstrings to understand what they do  

### The Pattern

```python
# 1. Create an agent
agent = Agent(model="...", deps_type=YourContextType)

# 2. Register tools
@agent.tool
def your_tool(ctx: RunContext[YourContextType]) -> str:
    """Clear description of what the tool does"""
    # Do something useful
    return result

# 3. Run the agent
response = agent.run_sync(prompt, deps=context)
```

### What's Next?

In **Step 3**, we'll add multiple tools and see how the agent intelligently chooses between them - or chooses to use *none* when its own knowledge is sufficient!

This is where agents really shine. 🌟

---

**⏸️ PAUSE: Questions before we continue to Step 3?**